# Text Generation after transformer block

Steps from phrase as input:
- tokenize input
- generate embeddings and positional
- transformer block (GPT)
- receives a matrix andbget last vector
- comnverts logits (vectore) to prob distriubution using softmax
- highest item is the tokenId to get word
- done

In [21]:
import torch
import torch.nn as nn
import tiktoken
import myllm.gpt as gpt


In [22]:
def generate_text_simple(model, idx, max_new_tokens, content_size):
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -content_size:]
        with torch.no_grad():
            logits = model(idx_cond)
            logits = logits[:, -1, :]
            probas = torch.softmax(logits, dim=-1)
            idx_next = torch.argmax(probas, dim=-1, keepdim=True)
            idx = torch.cat((idx, idx_next), dim=1)
    return idx

In [23]:
tokenizer = tiktoken.get_encoding("gpt2")

start_context = "Hello, I am"
encoded = tokenizer.encode(start_context)
print(encoded)

encoded_tensor = torch.tensor(encoded).unsqueeze(0)
print(encoded_tensor.shape)



[15496, 11, 314, 716]
torch.Size([1, 4])


In [24]:
torch.manual_seed(123)
model = gpt.GPTModel(gpt.GPT_CONFIG_124M)

logits = model(encoded_tensor)

print(logits.shape)
print(logits)

torch.Size([1, 4, 50257])
tensor([[[-0.2456, -0.3471,  0.0570,  ..., -0.1567,  0.3721, -0.3070],
         [ 0.9986, -0.1967, -0.3521,  ...,  0.4728,  0.7032, -0.0133],
         [ 1.0982,  0.4236,  0.5861,  ..., -0.2510,  0.8326, -0.0555],
         [-0.3278,  0.7634, -0.2148,  ...,  1.1910, -0.9351, -0.6408]]],
       grad_fn=<UnsafeViewBackward0>)


In [25]:
model.eval()

out = generate_text_simple(
    model=model,
    idx=encoded_tensor,
    max_new_tokens=6,
    content_size=gpt.GPT_CONFIG_124M["context_length"]
)

print(out)
print(len(out[0]))

tensor([[15496,    11,   314,   716, 27018,  7283, 46275, 41426, 33167, 33239]])
10


In [26]:
decoded_text= tokenizer.decode(out.squeeze(0).tolist())
print(decoded_text)

Hello, I am Feature IT snowballProtect youngstersMu
